In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(629))
random.seed(40)
random.shuffle(index)

train_index = index[0:440]
val_index = index[440:503]
test_index = index[503:629]

In [3]:
#dataimage_train = [dataimage[j] for j in train_index]
#print(dataimage_train)

In [4]:
lab = np.zeros((629,1)) #first AD:188 then CN:288
lab[228:629, 0] = 1 #MCI -- 1; CN -- 0
for i in range(0, 629):
    print(i, lab[i])

0 [0.]
1 [0.]
2 [0.]
3 [0.]
4 [0.]
5 [0.]
6 [0.]
7 [0.]
8 [0.]
9 [0.]
10 [0.]
11 [0.]
12 [0.]
13 [0.]
14 [0.]
15 [0.]
16 [0.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [0.]
27 [0.]
28 [0.]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [0.]
35 [0.]
36 [0.]
37 [0.]
38 [0.]
39 [0.]
40 [0.]
41 [0.]
42 [0.]
43 [0.]
44 [0.]
45 [0.]
46 [0.]
47 [0.]
48 [0.]
49 [0.]
50 [0.]
51 [0.]
52 [0.]
53 [0.]
54 [0.]
55 [0.]
56 [0.]
57 [0.]
58 [0.]
59 [0.]
60 [0.]
61 [0.]
62 [0.]
63 [0.]
64 [0.]
65 [0.]
66 [0.]
67 [0.]
68 [0.]
69 [0.]
70 [0.]
71 [0.]
72 [0.]
73 [0.]
74 [0.]
75 [0.]
76 [0.]
77 [0.]
78 [0.]
79 [0.]
80 [0.]
81 [0.]
82 [0.]
83 [0.]
84 [0.]
85 [0.]
86 [0.]
87 [0.]
88 [0.]
89 [0.]
90 [0.]
91 [0.]
92 [0.]
93 [0.]
94 [0.]
95 [0.]
96 [0.]
97 [0.]
98 [0.]
99 [0.]
100 [0.]
101 [0.]
102 [0.]
103 [0.]
104 [0.]
105 [0.]
106 [0.]
107 [0.]
108 [0.]
109 [0.]
110 [0.]
111 [0.]
112 [0.]
113 [0.]
114 [0.]
115 [0.]
116 [0.]
117 [0.]
118 [0.]
119 [0.]
120 [0.]
121 [0.]
122 [0.]
123

In [5]:
cn = 0.
mci = 0.
total_num = 0.
for i in lab:
    if(i == 1):
        mci += 1
    elif(i == 0):
        cn += 1
total_num = cn + mci
print(mci, cn, total_num)

401.0 228.0 629.0


In [6]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_CN_MCI_Feb4_3D.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((629,1)) #first AD:188 then CN:288
        self.lab[228:629, 0] = 1 #CN -- 0; MCI -- 1
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            self.img = self.img[train_index]
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            #for i in range(440):
                #for j in range(3):
                    #self.img[i][j] = self.img[i][j] + np.random.normal(0, 0.01, (121, 121))
                    #print(self.img[i][j].shape)
                    #count += 1
            #print(count)
            self.lab = self.lab[train_index]
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 
        return sample


In [7]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=2,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=2,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=2,
                        shuffle=True, num_workers=4)

print("finished")

great job dude
great job dude
great job dude
finished


In [8]:
#design the network configuration
import torch.nn as nn
import torch.nn.functional as F


#121 * 145 * 121 * 1
#60 * 72 * 60 * 2
#30 * 36 * 30 * 4
#15 * 18 * 15 * 8
#7 * 9 * 7 * 16
#3 * 4 * 3 * 32
#linear: 3 * 4 * 3 * 32 = 1152

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 16, 5, padding=2)
        self.conv5 = nn.Conv3d(16, 32, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(3 * 4 * 3 * 32, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 3 * 4 * 3 * 32)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net_3d = Net().cuda()

print("finished")

finished


In [9]:
import torch.optim as optim
import torch 
Weight = torch.tensor([total_num / cn, total_num / mci])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net_3d.parameters(), lr=0.01, momentum=0.9)

print("finished")

finished


In [10]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

loss_val_min = 1000000
#threshold = 20
count = 0
#count_big = 0
epoch = 50
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()
        images = np.expand_dims(images, 1)
        images = ToTensor(images).cuda()
        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_3d.train()

        # forward + backward + optimize
        outputs = net_3d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_3d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        #print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        images = np.expand_dims(images, 1)
        images = ToTensor(images).cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_3d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        #print(loss.item())
        #print(type(loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    if(total_val_loss < loss_val_min):
        loss_val_min = total_val_loss
        torch.save(net_3d, 'best_model_3D_Feb4_weighted_MCI_CN.pkl')
    #if(total_val_loss < min_total_loss):
        #min_total_loss = total_val_loss
        #save best model
        #torch.save(net_2d, 'best_model_Jan27_2d_3types.pkl')
    #if(count > threshold):
        #break
    #if(total_val_loss > 0.7):
        #count_big += 1
    #if(count_big > threshold):
        #break
    #if(total_val_loss < 0.67):
        #break

print("we're done training")

[1] loss: 0.680
[2] loss: 0.749
[3] loss: 0.645
[4] loss: 0.745
[5] loss: 0.682
[6] loss: 0.736
[7] loss: 0.685
[8] loss: 0.719
[9] loss: 0.707
[10] loss: 0.694
[11] loss: 0.711
[12] loss: 0.669
[13] loss: 0.704
[14] loss: 0.706
[15] loss: 0.744
[16] loss: 0.646
[17] loss: 0.748
[18] loss: 0.709
[19] loss: 0.707
[20] loss: 0.652
[21] loss: 0.706
[22] loss: 0.706
[23] loss: 0.733
[24] loss: 0.702
[25] loss: 0.673
[26] loss: 0.698
[27] loss: 0.706
[28] loss: 0.694
[29] loss: 0.701
[30] loss: 0.705
[31] loss: 0.691
[32] loss: 0.691
[33] loss: 0.701
[34] loss: 0.692
[35] loss: 0.693
[36] loss: 0.692
[37] loss: 0.696
[38] loss: 0.680
[39] loss: 0.670
[40] loss: 0.704
[41] loss: 0.645
[42] loss: 0.630
[43] loss: 0.720
[44] loss: 0.726
[45] loss: 0.731
[46] loss: 0.734
[47] loss: 0.736
[48] loss: 0.737
[49] loss: 0.737
[50] loss: 0.736
[51] loss: 0.734
[52] loss: 0.731
[53] loss: 0.728
[54] loss: 0.601
[55] loss: 0.723
[56] loss: 0.721
[57] loss: 0.615
[58] loss: 0.718
[59] loss: 0.717
[60] l

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[1] loss: 0.414
[2] loss: 0.843
[3] loss: 0.845
[4] loss: 0.410
[5] loss: 0.845
[6] loss: 0.844
[7] loss: 0.841
[8] loss: 0.419
[9] loss: 0.421
[10] loss: 0.420
[11] loss: 0.417
[12] loss: 0.843
[13] loss: 0.845
[14] loss: 0.408
[15] loss: 0.405
[16] loss: 0.853
[17] loss: 0.398
[18] loss: 0.859
[19] loss: 1.127
[20] loss: 1.119
[21] loss: 0.848
[22] loss: 0.839
[23] loss: 0.829
[24] loss: 1.032
[25] loss: 0.458
[26] loss: 0.795
[27] loss: 0.785
[28] loss: 0.502
[29] loss: 0.769
[30] loss: 0.524
[31] loss: 0.758
[32] loss: 0.874
[33] loss: 0.553
[34] loss: 0.561
[35] loss: 0.564
[36] loss: 0.741
[37] loss: 0.564
[38] loss: 0.846
[39] loss: 0.843
[40] loss: 0.570
[41] loss: 0.573
[42] loss: 0.571
[43] loss: 0.740
[44] loss: 0.563
[45] loss: 0.745
[46] loss: 0.747
[47] loss: 0.748
[48] loss: 0.550
[49] loss: 0.547
[50] loss: 0.539
[51] loss: 0.889
[52] loss: 0.526
[53] loss: 0.765
[54] loss: 0.767
[55] loss: 0.768
[56] loss: 0.514
[57] loss: 0.769
[58] loss: 0.770
[59] loss: 0.769
[60] l

[193] loss: 0.743
[194] loss: 0.850
[195] loss: 0.561
[196] loss: 0.560
[197] loss: 0.556
[198] loss: 0.548
[199] loss: 0.537
[200] loss: 0.761
[201] loss: 0.515
[202] loss: 0.774
[203] loss: 0.495
[204] loss: 0.786
[205] loss: 0.791
[206] loss: 0.977
[207] loss: 0.794
[208] loss: 0.476
[209] loss: 0.475
[210] loss: 0.472
[211] loss: 0.799
[212] loss: 0.463
[213] loss: 0.805
[214] loss: 0.454
[215] loss: 0.448
[216] loss: 0.440
[217] loss: 1.049
[218] loss: 0.828
[219] loss: 0.829
[220] loss: 0.427
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.829


Validation
[2] loss: 0.829


Validation
[3] loss: 1.059


Validation
[4] loss: 0.426


Validation
[5] loss: 0.426


Validation
[6] loss: 1.059


Validation
[7] loss: 1.059


Validation
[8] loss: 0.829


Validation
[9] loss: 0.829


Validation
[10] loss: 0.829


Validation
[11] loss: 1.059


Validation
[12] loss: 0.426


Validation
[13] loss: 1.059


Validation
[14] loss: 0.426


Validation
[15] loss: 0.830


Va

[109] loss: 0.695
[110] loss: 0.706
[111] loss: 0.688
[112] loss: 0.687
[113] loss: 0.723
[114] loss: 0.723
[115] loss: 0.718
[116] loss: 0.678
[117] loss: 0.690
[118] loss: 0.703
[119] loss: 0.692
[120] loss: 0.694
[121] loss: 0.694
[122] loss: 0.699
[123] loss: 0.691
[124] loss: 0.690
[125] loss: 0.696
[126] loss: 0.697
[127] loss: 0.698
[128] loss: 0.698
[129] loss: 0.709
[130] loss: 0.703
[131] loss: 0.694
[132] loss: 0.698
[133] loss: 0.697
[134] loss: 0.696
[135] loss: 0.694
[136] loss: 0.691
[137] loss: 0.695
[138] loss: 0.702
[139] loss: 0.688
[140] loss: 0.686
[141] loss: 0.708
[142] loss: 0.698
[143] loss: 0.709
[144] loss: 0.683
[145] loss: 0.683
[146] loss: 0.678
[147] loss: 0.669
[148] loss: 0.655
[149] loss: 0.639
[150] loss: 0.620
[151] loss: 0.600
[152] loss: 0.824
[153] loss: 0.563
[154] loss: 0.865
[155] loss: 0.755
[156] loss: 0.530
[157] loss: 0.902
[158] loss: 0.517
[159] loss: 0.770
[160] loss: 0.506
[161] loss: 0.776
[162] loss: 0.496
[163] loss: 0.489
[164] loss

...

Validation
[13] loss: 0.791


Validation
[14] loss: 0.476


Validation
[15] loss: 0.476


Validation
[16] loss: 0.791


Validation
[17] loss: 0.476


Validation
[18] loss: 0.791


Validation
[19] loss: 0.970


Validation
[20] loss: 0.476


Validation
[21] loss: 0.791


Validation
[22] loss: 0.791


Validation
[23] loss: 0.791


Validation
[24] loss: 0.791


Validation
[25] loss: 0.791


Validation
[26] loss: 0.970


Validation
[27] loss: 0.791


Validation
[28] loss: 0.476


Validation
[29] loss: 0.791


Validation
[30] loss: 0.791


Validation
[31] loss: 0.791


Validation
[32] loss: 0.476


[1] loss: 0.791
[2] loss: 0.467
[3] loss: 0.806
[4] loss: 1.018
[5] loss: 0.813
[6] loss: 0.814
[7] loss: 1.019
[8] loss: 1.008
[9] loss: 0.800
[10] loss: 0.476
[11] loss: 0.484
[12] loss: 0.488
[13] loss: 0.783
[14] loss: 0.781
[15] loss: 0.496
[16] loss: 0.938
[17] loss: 0.502
[18] loss: 0.504
[19] loss: 0.929
[20] loss: 0.506
[21] loss: 0.924
[22] loss: 0.769
[23] loss: 0.907
[24] loss: 0.760
[

[160] loss: 0.923
[161] loss: 0.770
[162] loss: 0.768
[163] loss: 0.903
[164] loss: 0.888
[165] loss: 0.544
[166] loss: 0.746
[167] loss: 0.564
[168] loss: 0.570
[169] loss: 0.737
[170] loss: 0.575
[171] loss: 0.574
[172] loss: 0.739
[173] loss: 0.567
[174] loss: 0.743
[175] loss: 0.744
[176] loss: 0.556
[177] loss: 0.747
[178] loss: 0.548
[179] loss: 0.752
[180] loss: 0.539
[181] loss: 0.757
[182] loss: 0.759
[183] loss: 0.760
[184] loss: 0.760
[185] loss: 0.889
[186] loss: 0.755
[187] loss: 0.751
[188] loss: 0.855
[189] loss: 0.837
[190] loss: 0.815
[191] loss: 0.605
[192] loss: 0.717
[193] loss: 0.711
[194] loss: 0.706
[195] loss: 0.701
[196] loss: 0.696
[197] loss: 0.691
[198] loss: 0.716
[199] loss: 0.685
[200] loss: 0.735
[201] loss: 0.738
[202] loss: 0.652
[203] loss: 0.682
[204] loss: 0.646
[205] loss: 0.751
[206] loss: 0.753
[207] loss: 0.680
[208] loss: 0.680
[209] loss: 0.746
[210] loss: 0.682
[211] loss: 0.683
[212] loss: 0.733
[213] loss: 0.685
[214] loss: 0.720
[215] loss

[75] loss: 0.342
[76] loss: 0.348
[77] loss: 0.352
[78] loss: 1.211
[79] loss: 0.360
[80] loss: 0.890
[81] loss: 0.884
[82] loss: 0.375
[83] loss: 0.379
[84] loss: 0.381
[85] loss: 0.871
[86] loss: 0.383
[87] loss: 0.382
[88] loss: 0.380
[89] loss: 0.376
[90] loss: 0.370
[91] loss: 0.363
[92] loss: 0.355
[93] loss: 0.346
[94] loss: 0.337
[95] loss: 0.327
[96] loss: 0.946
[97] loss: 0.956
[98] loss: 0.962
[99] loss: 0.304
[100] loss: 1.348
[101] loss: 1.345
[102] loss: 1.331
[103] loss: 0.315
[104] loss: 0.940
[105] loss: 1.271
[106] loss: 0.340
[107] loss: 0.905
[108] loss: 0.894
[109] loss: 0.882
[110] loss: 0.383
[111] loss: 0.859
[112] loss: 0.403
[113] loss: 0.842
[114] loss: 0.834
[115] loss: 0.431
[116] loss: 0.438
[117] loss: 1.030
[118] loss: 0.811
[119] loss: 0.804
[120] loss: 0.796
[121] loss: 0.788
[122] loss: 0.494
[123] loss: 0.503
[124] loss: 0.507
[125] loss: 0.508
[126] loss: 0.923
[127] loss: 0.919
[128] loss: 0.516
[129] loss: 0.764
[130] loss: 0.761
[131] loss: 0.886

Validation
[25] loss: 0.791


Validation
[26] loss: 0.477


Validation
[27] loss: 0.477


Validation
[28] loss: 0.477


Validation
[29] loss: 0.791


Validation
[30] loss: 0.477


Validation
[31] loss: 0.791


Validation
[32] loss: 0.477


[1] loss: 0.791
[2] loss: 0.966
[3] loss: 0.785
[4] loss: 0.494
[5] loss: 0.777
[6] loss: 0.774
[7] loss: 0.770
[8] loss: 0.519
[9] loss: 0.523
[10] loss: 0.524
[11] loss: 0.764
[12] loss: 0.764
[13] loss: 0.763
[14] loss: 0.525
[15] loss: 0.762
[16] loss: 0.761
[17] loss: 0.890
[18] loss: 0.755
[19] loss: 0.750
[20] loss: 0.854
[21] loss: 0.835
[22] loss: 0.587
[23] loss: 0.599
[24] loss: 0.722
[25] loss: 0.719
[26] loss: 0.716
[27] loss: 0.712
[28] loss: 0.745
[29] loss: 0.659
[30] loss: 0.668
[31] loss: 0.716
[32] loss: 0.697
[33] loss: 0.695
[34] loss: 0.696
[35] loss: 0.687
[36] loss: 0.689
[37] loss: 0.687
[38] loss: 0.685
[39] loss: 0.683
[40] loss: 0.745
[41] loss: 0.680
[42] loss: 0.752
[43] loss: 0.679
[44] loss: 0.750
[45] loss: 0.681
[46]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_3D_Feb4_weighted_MCI_CN.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
results_all_type = []
labels_all = []
for i_batch, sample_batched in enumerate(dataloader_test):
    #print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    images = np.expand_dims(images, 1)
    images = ToTensor(images).cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    #print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    #print("predicted:\n", results_type, "\nactual:\n", labels)

    results_all_type.append(results_type[0])
    results_all_type.append(results_type[1])
    labels_all.append(labels[0])
    labels_all.append(labels[1])
print(len(results_all_type))
print(len(labels_all))
CN_num = 0
MCI_num = 0
correct_sum_MCI = 0
correct_sum_CN = 0
for label in labels_all:
    if (label.float() == 1):
        MCI_num += 1
    if (label.float() == 0):
        CN_num += 1
print("actual", MCI_num, CN_num)

for i in range(len(results_all_type)):
    if(labels_all[i].float() == results_all_type[i].float()):
        if(results_all_type[i].float() == 1):
            correct_sum_MCI += 1
    if(labels_all[i].float() == results_all_type[i].float()):
        if(results_all_type[i].float() == 0):
            correct_sum_CN += 1
    total_sum += 1
    
print("correct_sum", correct_sum)
print("correct_sum_MCI", correct_sum_MCI)
print("correct_sum_CN", correct_sum_CN)
print(total_sum)


126
126
actual 80 46
correct_sum 0.0
correct_sum_MCI 80
correct_sum_CN 0
126.0
